In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

In [42]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3308450031&geoId=&keywords=Data%20Scientist&location=canada")

C:\Users\Hadi-PC\AppData\Local\Temp\ipykernel_15192\1197811035.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
driver.maximize_window()

actions = ActionChains(driver)

jobs_done = []

file = open('data.csv','w')
writer = csv.writer(file, delimiter=',')
writer.writerow(["title", "location", "company", "job description"])

while True:
    
    jobs_card = driver.find_elements(By.XPATH,"//a[@data-tracking-control-name='public_jobs_jserp-result_search-card']")

    for job in jobs_card:

        if job in jobs_done:
            continue
        else:
            jobs_done.append(job)

        actions.move_to_element(job).perform()

        job.click()

        time.sleep(0.5)

        location = WebDriverWait(driver,100).until(EC.visibility_of_element_located((By.XPATH,"//span[@class='topcard__flavor topcard__flavor--bullet']"))).text

        title = driver.find_element(By.XPATH,"//h2").text
    #     location = driver.find_element(By.XPATH,"//span[@class='topcard__flavor topcard__flavor--bullet']").text
        company = driver.find_element(By.XPATH,"//span[@class='topcard__flavor']").text

        try:
            #clicks on see more
            actions.move_to_element(WebDriverWait(driver,1).until(EC.presence_of_element_located((By.XPATH,"//button[@data-tracking-control-name='public_jobs_show-more-html-btn']")))).click().perform()
            time.sleep(1)
        except:pass

        details = driver.find_element(By.XPATH,"//div[@class='show-more-less-html__markup']").text

        writer.writerow([title, location, company, details])

        time.sleep(0.5)
